# Predicting RNA Half-Life from Epitranscriptomic and Structural Data using Convolutional Neural Networks


## **Import Samples**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd

# Specify the path to your text file
file_path = '/content/drive/MyDrive/Year 4/Fall/Machine Learning for Functional Genomics/project/final/OSD-569_m6a_counts_munged.txt'

# Read the text file into a pandas DataFrame
df_m6a = pd.read_csv(file_path, sep='\t')  # Assuming tab-separated, change 'sep' accordingly if needed

# Display the DataFrame
print(df_m6a)

In [ ]:
# Specify the path to your text file
file_path = '/content/drive/MyDrive/Year 4/Fall/Machine Learning for Functional Genomics/project/final/OSD-569_transcriptome_counts_munged.txt'

# Read the text file into a pandas DataFrame
df_transcripts = pd.read_csv(file_path, sep='\t')  # Assuming tab-separated, change 'sep' accordingly if needed

# Display the DataFrame
print(df_transcripts)


In [ ]:
# Specify the path to your text file
file_path = '/content/drive/MyDrive/Year 4/Fall/Machine Learning for Functional Genomics/project/final/OSD-569_sequences.txt'

# Read the text file into a pandas DataFrame
df_sequences = pd.read_csv(file_path, sep='\t')  # Assuming tab-separated, change 'sep' accordingly if needed

# Display the DataFrame
print(df_sequences)


In [ ]:
# Specify the path to your text file
file_path = '/content/drive/MyDrive/Year 4/Fall/Machine Learning for Functional Genomics/project/final/half_life.txt'

# Read the text file into a pandas DataFrame
df_half_life = pd.read_csv(file_path, sep='\t')  # Assuming tab-separated, change 'sep' accordingly if needed

# Display the DataFrame
print(df_half_life)


## Ensembl REST API Script - SEQUENCE CONTEXT IMPORT


In [ ]:
import requests
import pandas as pd

def fetch_transcript_sequence(ensembl_gene):
    server = "https://rest.ensembl.org"
    ext = f"/sequence/id/{ensembl_gene}?type=genomic"
    headers = {
        "Content-Type": "text/plain",
        "Accept": "text/plain"
    }
    response = requests.get(server + ext, headers=headers)
    if response.status_code == 200:
        return response.text
    else:
        print(f"Failed to fetch transcript sequence for {ensembl_gene}. Status code: {response.status_code}")
        return None

def extract_sequence_around_position(transcript_sequence, position):
    if transcript_sequence:
        position = int(position)
        sequence_start = max(0, position - 25)
        sequence_end = min(len(transcript_sequence), position + 25)
        sequence_around_position = transcript_sequence[sequence_start:sequence_end]
        return sequence_around_position
    else:
        return None

# Create a new column to store the sequences
df_sequences['Genomic Sequence'] = ''

# Remove numbers after the dot in ENSEMBL gene column
df_sequences['ENSEMBL gene'] = df_sequences['ENSEMBL gene'].apply(lambda x: x.split('.')[0])

# Loop through the DataFrame and fetch sequences
for index, row in df_sequences.iterrows():
    ensembl_gene = row['ENSEMBL gene']
    position_gene = row['position gene']

    # Fetch entire transcript sequence for the ENSEMBL gene ID
    transcript_sequence = fetch_transcript_sequence(ensembl_gene)

    if transcript_sequence:
        # Extract sequence around the given position within the transcript sequence
        sequence_around_position = extract_sequence_around_position(transcript_sequence, position_gene)
        if sequence_around_position:
            # Add the extracted sequence to the DataFrame
            df_sequences.at[index, 'Genomic Sequence'] = sequence_around_position
        else:
            print(f"Failed to extract sequence around position {position_gene} from the transcript sequence of {ensembl_gene}.")
    else:
        print(f"Transcript sequence not found for {ensembl_gene}.")

# Display the updated DataFrame with genomic sequences
print(df_sequences)

In [ ]:
# Export DataFrame to a tab-separated text file
file_path = '/content/drive/MyDrive/Year 4/Fall/Machine Learning for Functional Genomics/project/final/OSD-569_sequences_filled.txt'
df_sequences.to_csv(file_path, sep='\t', index=False)

In [ ]:
import requests
import pandas as pd

def fetch_transcript_sequence(ensembl_gene):
    server = "https://rest.ensembl.org"
    ext = f"/sequence/id/{ensembl_gene}?type=cdna"
    headers = {
        "Content-Type": "text/plain",
        "Accept": "text/plain"
    }
    response = requests.get(server + ext, headers=headers)
    if response.status_code == 200:
        return response.text
    else:
        print(f"Failed to fetch transcript sequence for {ensembl_gene}. Status code: {response.status_code}")
        return None

def extract_sequence_around_position(transcript_sequence, position):
    if transcript_sequence:
        position = int(position)
        sequence_start = 0
        sequence_end = len(transcript_sequence)
        sequence_around_position = transcript_sequence[sequence_start:sequence_end]
        return sequence_around_position
    else:
        return None

# Specify the path to your text file
file_path = '/Users/theo/Downloads/OSD-569_sequences.txt'

# Read the text file into a pandas DataFrame
df_sequences = pd.read_csv(file_path, sep='\t')  # Assuming tab-separated, change 'sep' accordingly if needed

# Display the DataFrame
print(df_sequences)

# Create a new column to store the sequences
df_sequences['Genomic Sequence'] = ''

# Remove numbers after the dot in ENSEMBL gene column
df_sequences['ENSEMBL transcript'] = df_sequences['ENSEMBL transcript'].apply(lambda x: x.split('.')[0])

# Open the file to write sequences
output_file_path = '/Users/theo/Downloads/OSD-569_sequences_complete.txt'
with open(output_file_path, 'w') as output_file:
    sequence_counter = 0
    save_interval = 100  # Define the interval to save the file
    start_index = 20400  # Specify the index to start fetching sequences

    # Loop through the DataFrame from the specified starting index
    for index, row in df_sequences.iloc[start_index:].iterrows():
        ensembl_gene = row['ENSEMBL transcript']
        position_gene = row['position transcript']

        # Fetch entire transcript sequence for the ENSEMBL gene ID
        transcript_sequence = fetch_transcript_sequence(ensembl_gene)
        print(index)
        if transcript_sequence:
            # Extract sequence around the given position within the transcript sequence
            sequence_around_position = extract_sequence_around_position(transcript_sequence, position_gene)
            if sequence_around_position:
                # Write the sequence to the file immediately after fetching
                output_file.write(f"Sequence for index {index}:{sequence_around_position}\n")
                sequence_counter += 1

                # Save the file at regular intervals
                if sequence_counter % save_interval == 0:
                    output_file.flush()  # Flush buffer to write to file immediately
                    print(f"File saved after fetching {sequence_counter} sequences.")
            else:
                print(f"Failed to extract sequence around position {position_gene} from the transcript sequence of {ensembl_gene}.")
        else:
            print(f"Transcript sequence not found for {ensembl_gene}.")

# Confirm completion
print("Sequences written to file.")

In [ ]:
import requests
import pandas as pd

def fetch_transcript_sequence(ensembl_gene):
    server = "https://rest.ensembl.org"
    ext = f"/sequence/id/{ensembl_gene}?type=cdna"
    headers = {
        "Content-Type": "text/plain",
        "Accept": "text/plain"
    }
    response = requests.get(server + ext, headers=headers)
    if response.status_code == 200:
        return response.text
    else:
        print(f"Failed to fetch transcript sequence for {ensembl_gene}. Status code: {response.status_code}")
        return None

def calculate_position_percentage(transcript_sequence, position):
    if transcript_sequence:
        total_length = len(transcript_sequence)
        position_percentage = (int(position) / total_length) * 100
        return position_percentage
    else:
        return None

# Specify the path to your text file
file_path = '/Users/theo/Downloads/OSD-569_sequences.txt'  # Replace with your file path

# Read the text file into a pandas DataFrame
df_sequences = pd.read_csv(file_path, sep='\t')  # Uncomment and adjust 'sep' accordingly if needed

# Create a new column to store the position percentages
df_sequences['Position Percentage'] = None

# Remove numbers after the dot in ENSEMBL gene column
df_sequences['ENSEMBL transcript'] = df_sequences['ENSEMBL transcript'].apply(lambda x: x.split('.')[0])

# Output file path
output_file_path = '/Users/theo/Downloads/OSD-569_positions_filled.txt'

# Open the file to write sequences
with open(output_file_path, 'w') as output_file:
    sequence_counter = 0
    save_interval = 100  # Define the interval to save the file
    start_index = 0  # Specify the index to start fetching sequences

    # Loop through the DataFrame from the specified starting index
    for index, row in df_sequences.iloc[start_index:].iterrows():
        ensembl_gene = row['ENSEMBL transcript']
        position_gene = row['position transcript']

        # Fetch entire transcript sequence for the ENSEMBL gene ID
        transcript_sequence = fetch_transcript_sequence(ensembl_gene)
        print(index)
        if transcript_sequence:
            # Calculate the position percentage
            position_percentage = calculate_position_percentage(transcript_sequence, position_gene)
            if position_percentage is not None:
                # Write the percentage to the file immediately after calculating
                output_file.write(f"Percentage position for index {index}: {position_percentage}\n")
                sequence_counter += 1

                # Save the file at regular intervals
                if sequence_counter % save_interval == 0:
                    output_file.flush()  # Flush buffer to write to file immediately
                    print(f"File saved after processing {sequence_counter} entries.")
            else:
                print(f"Failed to calculate position percentage for {ensembl_gene} at position {position_gene}.")
        else:
            print(f"Transcript sequence not found for {ensembl_gene}.")

# Confirm completion
output_file.flush()
print("Position percentages written to file.")

## Lasso Regression Script (m6a => gene expression prediction)


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# Load m6A data
m6a_data = df_m6a

# Load transcriptomic data
transcriptomic_data = df_transcripts

# Replace 'NA' values with NaN for easier handling
m6a_data.replace('NA', pd.NA, inplace=True)

# Calculate average across columns C11 to C47 for each row
m6a_data['m6a_avg'] = m6a_data.iloc[:, 4:33].mean(axis=1)

# Merge m6A data with transcriptomic data based on the common gene identifiers
merged_data = pd.merge(m6a_data, transcriptomic_data, left_on='ENSEMBL gene', right_on='Genes')

# Select relevant columns for modeling
features = merged_data[['m6a_avg']]
target = merged_data[['C47_y']]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Initialize a linear regression model
model = LinearRegression()

# Train the model
model.fit(X_train, y_train)

# Evaluate the model
test_predictions = model.predict(X_test)
mse = mean_squared_error(y_test, test_predictions)
r_squared = r2_score(y_test, test_predictions)

print("Mean Squared Error:", mse)
print("R-squared:", r_squared)

# Predict transcriptomic values for new data
# Assuming 'new_m6a_data' contains new m6A values for prediction
# new_m6a_data = pd.read_csv('path_to_new_m6a_data.csv')  # Replace 'path_to_new_m6a_data.csv' with your file path
# new_m6a_data.replace('NA', pd.NA, inplace=True)
# new_m6a_data['m6a_avg'] = new_m6a_data.iloc[:, 4:50].mean(axis=1)

predicted_transcriptomic_values = model.predict(new_m6a_data[['m6a_avg']])

## Regular Neural Network (m6a => gene expression prediction)


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input

# Load m6A data
m6a_data = df_m6a

# Replace 'NA' values with NaN for easier handling
m6a_data.replace('NA', pd.NA, inplace=True)

# Average 'm6A_values' across columns C11 to C47
m6a_data['m6a_avg'] = m6a_data.iloc[:, 4:33].mean(axis=1)

# Select relevant columns for modeling (averaged m6a values and positions)
features = m6a_data[['m6a_avg']]

# Load transcriptomic data
transcriptomic_data = df_transcripts

# Average 'Transcriptomic_value' across columns C11 to C47
transcriptomic_data['Transcriptomic_avg'] = transcriptomic_data.iloc[:, 2:31].mean(axis=1)

# Log normalization of 'Transcriptomic_avg' values
transcriptomic_data['Transcriptomic_avg_log'] = np.log1p(transcriptomic_data['Transcriptomic_avg'])

# Merge m6A data with transcriptomic data based on the common gene identifiers
merged_data = pd.merge(m6a_data, transcriptomic_data, left_on='ENSEMBL gene', right_on='Genes')

# Separate features and target
X = features.values
y = merged_data['Transcriptomic_avg_log'].values
print(X)
print(y)
# Scale the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Get unique genes for train-test split
unique_genes = merged_data['ENSEMBL gene'].unique()
train_genes, test_genes = train_test_split(unique_genes, test_size=0.2, random_state=42)

# Filter data for train-test split based on gene
train_data = merged_data[merged_data['ENSEMBL gene'].isin(train_genes)]
test_data = merged_data[merged_data['ENSEMBL gene'].isin(test_genes)]

# Train-test split
X_train = X_scaled[train_data.index]
y_train = train_data['Transcriptomic_avg_log'].values
X_test = X_scaled[test_data.index]
y_test = test_data['Transcriptomic_avg_log'].values

# Build the neural network model
model = Sequential([
    Input(shape=(X_train.shape[1],)),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(16, activation='relu'),
    Dense(8, activation='relu'),
    Dense(1)  # Output layer
])

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2)

# Evaluate the model
test_predictions = model.predict(X_test)
mse = mean_squared_error(y_test, test_predictions)
r_squared = r2_score(y_test, test_predictions)

print("Mean Squared Error:", mse)
print("R-squared:", r_squared)


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input

# Assuming the previous code for data preparation and model training has been executed

# Generate ten random indices from the test set
num_predictions = 10
random_indices = np.random.choice(len(y_test), size=num_predictions, replace=False)

# Get the actual values and corresponding predictions
actual_values = y_test[random_indices]
predicted_values = test_predictions[random_indices].flatten()

# Calculate percentage error for each prediction
percentage_errors = np.abs((predicted_values - actual_values) / actual_values) * 100

# Create a DataFrame to display the information
data = {
    'Actual Values (Log Value)': actual_values,
    'Predicted Values (Log Value)': predicted_values,
    'Percentage Error (%)': percentage_errors
}
prediction_comparison = pd.DataFrame(data)

# Display the table
print(prediction_comparison)

## Conventional Neural Network (m6a => gene expression prediction)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten
from tensorflow.keras.layers import LSTM, Bidirectional
from tensorflow.keras.layers import Dropout

# Assuming df_sequences contains sequence data in the specified format
# Let's consider 'sequence' column contains DNA sequences

# Load merged data (contains m6a data)
# Assuming merged_data contains the merged data of m6a with other features
# Replace this with your actual merged data

# Load transcriptomic data (Assuming df_transcriptomic contains transcriptomic data)
# Replace this with your actual transcriptomic data
# Load transcriptomic data
transcriptomic_data = df_transcripts

# Average 'Transcriptomic_value' across columns C11 to C47
transcriptomic_data['Transcriptomic_avg'] = transcriptomic_data.iloc[:, 2:31].mean(axis=1)

# Log normalization of 'Transcriptomic_avg' values
transcriptomic_data['Transcriptomic_avg_log'] = np.log1p(transcriptomic_data['Transcriptomic_avg'])

# Merge m6A data with transcriptomic data based on the common gene identifiers
merged_data = pd.merge(m6a_data, transcriptomic_data, left_on='ENSEMBL gene', right_on='Genes')

# Merge sequence data with existing merged data based on the common gene identifiers
# Assuming 'ENSEMBL gene' is the common identifier between merged_data and df_sequences
merged_data_with_sequence = pd.merge(merged_data, df_sequences, on=['ENSEMBL gene', 'position gene'], how='left')

# Extracting Gene IDs without numbers after the dot
merged_data_with_sequence['Gene_ID'] = merged_data_with_sequence['ENSEMBL gene'].str.split('.', expand=True)[0]

# Merge the modified DataFrame with df_half_life based on the Gene ID
merged_with_half_life = pd.merge(merged_data_with_sequence, df_half_life, left_on='Gene_ID', right_on='Ensembl Gene Id')

# Use 'half-life (PC1)' as the target instead of transcriptomic outputs
X = merged_data_with_sequence[['m6a_avg', 'sequences']]
X['sequences'].fillna('', inplace=True)

# Select relevant columns for modeling (averaged m6a values, positions, and sequence)
features_with_sequence = merged_data_with_sequence[['m6a_avg', 'sequences']]

# Handling missing sequence values (if any)
features_with_sequence['sequences'].fillna('', inplace=True)

# Define a mapping for one-hot encoding
base_to_idx = {'A': 0, 'C': 1, 'G': 2, 'T': 3}

# Define a function to perform one-hot encoding for DNA sequences
def one_hot_encode_sequence(sequence, sequence_length):
    encoded_sequence = np.zeros((sequence_length, 4))  # 4 for A, C, G, T
    for i, base in enumerate(sequence[:sequence_length]):
        if base in base_to_idx:
            encoded_sequence[i, base_to_idx[base]] = 1
    return encoded_sequence

# Define the sequence length (adjust as needed based on your requirement)
sequence_length = 50  # Assuming a sequence length of 100 bases

# Apply one-hot encoding to the 'sequence' column
features_with_sequence['one_hot_encoded_sequence'] = features_with_sequence['sequences'].apply(
    lambda seq: one_hot_encode_sequence(seq, sequence_length)
)

# Prepare features (X) and target (y) for modeling
X_sequence = np.array(features_with_sequence['one_hot_encoded_sequence'].tolist())
X_m6a = features_with_sequence['m6a_avg'].values.reshape(-1, 1)
X_sequence = X_sequence.reshape(X_sequence.shape[0], -1)
X = np.concatenate((X_sequence, X_m6a), axis=1)

# Extract transcriptomic target values
y_transcriptomic = merged_data_with_sequence['Transcriptomic_avg_log'].values

# Assuming 'Transcriptomic_avg_log' was the column name for transcriptomic values
# Concatenate transcriptomic target values with existing y values
y = np.concatenate((y, y_transcriptomic), axis=0)

# Limit the number of data points to the first 15700
X = X[:11000]
y_transcriptomic = y_transcriptomic[:11000]

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y_transcriptomic, test_size=0.2, random_state=42)

# Scaling the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(X_train_scaled.shape)

print(X_train_scaled.shape[0])

# Assuming sequence_length and features are known
sequence_length = 201
features = 1

model = Sequential([
    Input(shape=(sequence_length, features)),  # Input shape adjusted for sequence data
    Conv1D(32, kernel_size=3, activation='relu'),
    Dropout(0.3),  # Adding dropout to prevent overfitting
    Conv1D(64, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Dropout(0.3),  # Adding dropout to prevent overfitting
    Conv1D(128, kernel_size=3, activation='relu'),
    Dropout(0.3),  # Adding dropout to prevent overfitting
    Conv1D(256, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Dropout(0.3),  # Adding dropout to prevent overfitting
    Flatten(),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(1)  # Output layer
])

import matplotlib.pyplot as plt

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
history = model.fit(X_train_scaled, y_train, epochs=50, batch_size=32, validation_split=0.2)

# Extract loss values from the training history
train_loss = history.history['loss']
val_loss = history.history['val_loss']

# Plotting training and validation loss across epochs
epochs = range(1, len(train_loss) + 1)
plt.plot(epochs, train_loss, 'bo', label='Training Loss')
plt.plot(epochs, val_loss, 'r', label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

# Evaluate the model
test_predictions = model.predict(X_test_scaled)
mse = mean_squared_error(y_test, test_predictions)
r_squared = r2_score(y_test, test_predictions)

print("Mean Squared Error:", mse)
print("R-squared:", r_squared)

## Conventional Neural Network (m6a => mRNA decay)


In [ ]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten
from tensorflow.keras.layers import LSTM, Bidirectional
from tensorflow.keras.layers import Dropout

# Load m6A data
m6a_data = df_m6a

# Replace 'NA' values with NaN for easier handling
m6a_data.replace('NA', pd.NA, inplace=True)

# Average 'm6A_values' across columns C11 to C47
m6a_data['m6a_avg'] = m6a_data.iloc[:, 4:33].mean(axis=1)

# Select relevant columns for modeling (averaged m6a values and positions)
features = m6a_data[['m6a_avg']]

# Load transcriptomic data
transcriptomic_data = df_transcripts

# Average 'Transcriptomic_value' across columns C11 to C47
transcriptomic_data['Transcriptomic_avg'] = transcriptomic_data.iloc[:, 2:31].mean(axis=1)

# Log normalization of 'Transcriptomic_avg' values
transcriptomic_data['Transcriptomic_avg_log'] = np.log1p(transcriptomic_data['Transcriptomic_avg'])

# Merge m6A data with transcriptomic data based on the common gene identifiers
merged_data = pd.merge(m6a_data, transcriptomic_data, left_on='ENSEMBL gene', right_on='Genes')

# Merge sequence data with existing merged data based on the common gene identifiers
# Assuming 'ENSEMBL gene' is the common identifier between merged_data and df_sequences
merged_data_with_sequence = pd.merge(merged_data, df_sequences, on=['ENSEMBL gene', 'position gene'], how='left')

# Extracting Gene IDs without numbers after the dot
merged_data_with_sequence['Gene_ID'] = merged_data_with_sequence['ENSEMBL gene'].str.split('.', expand=True)[0]

# Merge the modified DataFrame with df_half_life based on the Gene ID
merged_with_half_life = pd.merge(merged_data_with_sequence, df_half_life, left_on='Gene_ID', right_on='Ensembl Gene Id')

# Use 'half-life (PC1)' as the target instead of transcriptomic outputs
X = merged_with_half_life[['m6a_avg', 'sequences']]
X['sequences'].fillna('', inplace=True)

# Step 1: Filter Rows with More Than Twelve Non-NA m6A Values
# Define the columns to check for non-NA m6a values
m6a_columns = [
    'C11_x', 'C12_x', 'C13_x', 'C14_x', 'C15_x', 'C16_x', 'C17_x',
    'C21_x', 'C22_x', 'C23_x', 'C24_x', 'C25_x', 'C26_x', 'C27_x',
    'C31_x', 'C32_x', 'C33_x', 'C34_x', 'C35_x', 'C36_x', 'C37_x',
    'C41_x', 'C42_x', 'C43_x', 'C44_x', 'C45_x', 'C46_x', 'C47_x'
]

# Filter rows with more than twelve non-NA values
merged_with_half_life_filtered = merged_with_half_life.dropna(subset=m6a_columns, thresh=12)

# Step 2: Calculate Standard Deviation and Add as New Column
merged_with_half_life_filtered['m6a_stddev'] = merged_with_half_life_filtered[m6a_columns].std(axis=1)

# Select relevant columns for modeling (averaged m6a values, positions, and sequence)
features_with_sequence = merged_with_half_life_filtered[['m6a_avg', 'm6a_stddev', 'sequences']]

# Handling missing sequence values (if any)
features_with_sequence['sequences'].fillna('', inplace=True)

# Define a mapping for one-hot encoding
base_to_idx = {'A': 0, 'C': 1, 'G': 2, 'T': 3}

# Define a function to perform one-hot encoding for DNA sequences
def one_hot_encode_sequence(sequence, sequence_length):
    encoded_sequence = np.zeros((sequence_length, 4))  # 4 for A, C, G, T
    for i, base in enumerate(sequence[:sequence_length]):
        if base in base_to_idx:
            encoded_sequence[i, base_to_idx[base]] = 1
    return encoded_sequence

# Define the sequence length (adjust as needed based on your requirement)
sequence_length = 50  # Assuming a sequence length of 100 bases

# Apply one-hot encoding to the 'sequence' column
features_with_sequence['one_hot_encoded_sequence'] = features_with_sequence['sequences'].apply(
    lambda seq: one_hot_encode_sequence(seq, sequence_length)
)

# Prepare features (X) and target (y) for modeling
X_sequence = np.array(features_with_sequence['one_hot_encoded_sequence'].tolist())
X_m6a = features_with_sequence['m6a_avg'].values.reshape(-1, 1)
X_m6a_stddev = features_with_sequence['m6a_stddev'].values.reshape(-1, 1)
X_sequence = X_sequence.reshape(X_sequence.shape[0], -1)
X = np.concatenate((X_sequence, X_m6a, X_m6a_stddev), axis=1)

# Extract transcriptomic target values
y_transcriptomic = merged_with_half_life['half-life (PC1)'].values

# Assuming 'Transcriptomic_avg_log' was the column name for transcriptomic values
# Concatenate transcriptomic target values with existing y values
y = np.concatenate((y, y_transcriptomic), axis=0)

# Limit the number of data points to the first 2000
X = X[:2000]
y_transcriptomic = y_transcriptomic[:2000]

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y_transcriptomic, test_size=0.2, random_state=42)

# Scaling the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(X_train_scaled.shape)

print(X_train_scaled.shape[0])

# Assuming sequence_length and features are known
sequence_length = 202
features = 1

model = Sequential([
    Input(shape=(sequence_length, features)),  # Input shape adjusted for sequence data
    Conv1D(32, kernel_size=3, activation='relu'),
    Dropout(0.3),  # Adding dropout to prevent overfitting
    Conv1D(64, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Dropout(0.3),  # Adding dropout to prevent overfitting
    Conv1D(128, kernel_size=3, activation='relu'),
    Dropout(0.3),  # Adding dropout to prevent overfitting
    Conv1D(256, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Dropout(0.3),  # Adding dropout to prevent overfitting
    Flatten(),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(1)  # Output layer
])

import matplotlib.pyplot as plt

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
history = model.fit(X_train_scaled, y_train, epochs=50, batch_size=32, validation_split=0.2)

# Extract loss values from the training history
train_loss = history.history['loss']
val_loss = history.history['val_loss']

# Plotting training and validation loss across epochs
epochs = range(1, len(train_loss) + 1)
plt.plot(epochs, train_loss, 'bo', label='Training Loss')
plt.plot(epochs, val_loss, 'r', label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

# Evaluate the model
test_predictions = model.predict(X_test_scaled)
mse = mean_squared_error(y_test, test_predictions)
r_squared = r2_score(y_test, test_predictions)

print("Mean Squared Error:", mse)
print("R-squared:", r_squared)

In [ ]:
for col in merged_with_half_life.columns:
    print(col)
print(merged_with_half_life)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input

# Assuming the previous code for data preparation and model training has been executed

# Generate ten random indices from the test set
num_predictions = 10
random_indices = np.random.choice(len(y_test), size=num_predictions, replace=False)

# Get the actual values and corresponding predictions
actual_values = y_test[random_indices]
predicted_values = test_predictions[random_indices].flatten()

# Calculate percentage error for each prediction
percentage_errors = np.abs((predicted_values - actual_values) / actual_values) * 100

# Create a DataFrame to display the information
data = {
    'Actual Values (Log Value)': actual_values,
    'Predicted Values (Log Value)': predicted_values,
    'Percentage Error (%)': percentage_errors
}
prediction_comparison = pd.DataFrame(data)

# Display the table
print(prediction_comparison)

# Calculate R-squared value
r_squared = r2_score(y_test, test_predictions)

# Scatter plot of predicted vs actual values
plt.subplot(1, 2, 2)
plt.scatter(y_test, test_predictions)
plt.plot(y_test, y_test, color='red', label='Ideal Prediction')
plt.title('Predicted vs Actual Values')
plt.xlabel('Actual Values (Log Value)')
plt.ylabel('Predicted Values (Log Value)')
plt.legend()

# Include R-squared value as text on the plot
plt.text(0.1, 0.9, f'R-squared = {r_squared:.4f}', ha='center', va='center', transform=plt.gca().transAxes, bbox=dict(facecolor='white', alpha=0.5))

plt.tight_layout()
plt.show()

## Conventional Neural Network (m6a AVERAGE / STDDEV + POSITION => mRNA decay) V2


In [ ]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten
from tensorflow.keras.layers import LSTM, Bidirectional
from tensorflow.keras.layers import Dropout

# Load m6A data
m6a_data = df_m6a

# Replace 'NA' values with NaN for easier handling
m6a_data.replace('NA', pd.NA, inplace=True)

# Average 'm6A_values' across columns C11 to C47
m6a_data['m6a_avg'] = m6a_data.iloc[:, 4:33].mean(axis=1)

# Select relevant columns for modeling (averaged m6a values and positions)
features = m6a_data[['m6a_avg']]

# Load transcriptomic data
transcriptomic_data = df_transcripts

# Average 'Transcriptomic_value' across columns C11 to C47
transcriptomic_data['Transcriptomic_avg'] = transcriptomic_data.iloc[:, 2:31].mean(axis=1)

# Log normalization of 'Transcriptomic_avg' values
transcriptomic_data['Transcriptomic_avg_log'] = np.log1p(transcriptomic_data['Transcriptomic_avg'])

# Merge m6A data with transcriptomic data based on the common gene identifiers
merged_data = pd.merge(m6a_data, transcriptomic_data, left_on='ENSEMBL gene', right_on='Genes')

# Merge sequence data with existing merged data based on the common gene identifiers
# Assuming 'ENSEMBL gene' is the common identifier between merged_data and df_sequences
merged_data_with_sequence = pd.merge(merged_data, df_sequences, on=['ENSEMBL gene', 'position gene'], how='left')

# Extracting Gene IDs without numbers after the dot
merged_data_with_sequence['Gene_ID'] = merged_data_with_sequence['ENSEMBL gene'].str.split('.', expand=True)[0]

# Merge the modified DataFrame with df_half_life based on the Gene ID
merged_with_half_life = pd.merge(merged_data_with_sequence, df_half_life, left_on='Gene_ID', right_on='Ensembl Gene Id')

# Drop rows with NaN in the 'position' column
merged_with_half_life = merged_with_half_life.dropna(subset=['position'])

# Use 'half-life (PC1)' as the target instead of transcriptomic outputs
X = merged_with_half_life[['m6a_avg', 'sequences', 'position']]
X['sequences'].fillna('', inplace=True)

# Step 1: Filter Rows with More Than Twelve Non-NA m6A Values
# Define the columns to check for non-NA m6a values
m6a_columns = [
    'C11_x', 'C12_x', 'C13_x', 'C14_x', 'C15_x', 'C16_x', 'C17_x',
    'C21_x', 'C22_x', 'C23_x', 'C24_x', 'C25_x', 'C26_x', 'C27_x',
    'C31_x', 'C32_x', 'C33_x', 'C34_x', 'C35_x', 'C36_x', 'C37_x',
    'C41_x', 'C42_x', 'C43_x', 'C44_x', 'C45_x', 'C46_x', 'C47_x'
]

# Filter rows with more than twelve non-NA values
merged_with_half_life_filtered = merged_with_half_life.dropna(subset=m6a_columns, thresh=12)

# Step 2: Calculate Standard Deviation and Add as New Column
merged_with_half_life_filtered['m6a_stddev'] = merged_with_half_life_filtered[m6a_columns].std(axis=1)

# Select relevant columns for modeling (averaged m6a values, positions, and sequence)
features_with_sequence = merged_with_half_life[['m6a_avg', 'position', 'sequences']]

# Handling missing sequence values (if any)
features_with_sequence['sequences'].fillna('', inplace=True)

# Define a mapping for one-hot encoding
base_to_idx = {'A': 0, 'C': 1, 'G': 2, 'T': 3}

# Define a function to perform one-hot encoding for DNA sequences
def one_hot_encode_sequence(sequence, sequence_length):
    encoded_sequence = np.zeros((sequence_length, 4))  # 4 for A, C, G, T
    for i, base in enumerate(sequence[:sequence_length]):
        if base in base_to_idx:
            encoded_sequence[i, base_to_idx[base]] = 1
    return encoded_sequence

# Define the sequence length (adjust as needed based on your requirement)
sequence_length = 50  # Assuming a sequence length of 50 bases

# Apply one-hot encoding to the 'sequence' column
features_with_sequence['one_hot_encoded_sequence'] = features_with_sequence['sequences'].apply(
    lambda seq: one_hot_encode_sequence(seq, sequence_length)
)

# Prepare features (X) and target (y) for modeling
X_sequence = np.array(features_with_sequence['one_hot_encoded_sequence'].tolist())
X_m6a = features_with_sequence['m6a_avg'].values.reshape(-1, 1)
X_m6a_pos = features_with_sequence['position'].values.reshape(-1, 1)
# X_m6a_stddev = features_with_sequence['m6a_stddev'].values.reshape(-1, 1)
X_sequence = X_sequence.reshape(X_sequence.shape[0], -1)
X = np.concatenate((X_sequence, X_m6a, X_m6a_pos), axis=1)

# Extract transcriptomic target values
y_transcriptomic = merged_with_half_life['half-life (PC1)'].values

# Assuming 'Transcriptomic_avg_log' was the column name for transcriptomic values
# Concatenate transcriptomic target values with existing y values
# y = np.concatenate((y, y_transcriptomic), axis=0)

# Limit the number of data points to the first 2000
X = X[:4400]
y_transcriptomic = y_transcriptomic[:4400]

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y_transcriptomic, test_size=0.2, random_state=42)

# Scaling the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(X_train_scaled.shape)

print(X_train_scaled.shape[0])

# Assuming sequence_length and features are known
sequence_length = 202
features = 1

model = Sequential([
    Input(shape=(sequence_length, features)),  # Input shape adjusted for sequence data
    Conv1D(32, kernel_size=3, activation='relu'),
    Dropout(0.3),  # Adding dropout to prevent overfitting
    Conv1D(64, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Dropout(0.3),  # Adding dropout to prevent overfitting
    Conv1D(128, kernel_size=3, activation='relu'),
    Dropout(0.3),  # Adding dropout to prevent overfitting
    Conv1D(256, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Dropout(0.3),  # Adding dropout to prevent overfitting
    Flatten(),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(1)  # Output layer
])

import matplotlib.pyplot as plt

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
history = model.fit(X_train_scaled, y_train, epochs=50, batch_size=32, validation_split=0.2)

# Extract loss values from the training history
train_loss = history.history['loss']
val_loss = history.history['val_loss']

# Plotting training and validation loss across epochs
epochs = range(1, len(train_loss) + 1)
plt.plot(epochs, train_loss, 'bo', label='Training Loss')
plt.plot(epochs, val_loss, 'r', label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

# Evaluate the model
test_predictions = model.predict(X_test_scaled)
mse = mean_squared_error(y_test, test_predictions)
r_squared = r2_score(y_test, test_predictions)

print("Mean Squared Error:", mse)
print("R-squared:", r_squared)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input, Conv1D, MaxPooling1D, Flatten, Dropout
import matplotlib.pyplot as plt

# Load and preprocess data
m6a_data = df_m6a.replace('NA', pd.NA)
m6a_data['m6a_avg'] = m6a_data.iloc[:, 4:33].mean(axis=1)

transcriptomic_data = df_transcripts
transcriptomic_data['Transcriptomic_avg_log'] = np.log1p(transcriptomic_data.iloc[:, 2:31].mean(axis=1))

# Merge datasets
merged_data = pd.merge(m6a_data, transcriptomic_data, left_on='ENSEMBL gene', right_on='Genes')
merged_data = pd.merge(merged_data, df_sequences, on=['ENSEMBL gene', 'position gene'], how='left')
merged_data['Gene_ID'] = merged_data['ENSEMBL gene'].str.split('.', expand=True)[0]
merged_data = pd.merge(merged_data, df_half_life, left_on='Gene_ID', right_on='Ensembl Gene Id')

# Drop rows with NaN in the 'position' column
merged_data = merged_data.dropna(subset=['position'])

# Prepare features and target
features = merged_data[['m6a_avg', 'position', 'sequences']]
features['sequences'].fillna('', inplace=True)

# One-hot encoding for sequences
base_to_idx = {'A': 0, 'C': 1, 'G': 2, 'T': 3}
sequence_length = 50
def one_hot_encode_sequence(sequence):
    encoded = np.zeros((sequence_length, 4))
    for i, base in enumerate(sequence[:sequence_length]):
        if base in base_to_idx:
            encoded[i, base_to_idx[base]] = 1
    return encoded

features['encoded_sequences'] = features['sequences'].apply(one_hot_encode_sequence)

# Prepare model input
X_sequence = np.array(features['encoded_sequences'].tolist())
X_m6a = features['m6a_avg'].values.reshape(-1, 1)
X_position = features['position'].values.reshape(-1, 1)
X = np.concatenate((X_sequence.reshape(X_sequence.shape[0], -1), X_m6a, X_position), axis=1)

# Prepare target
y = merged_data['half-life (PC1)'].values

X = X[:50000]
y = y[:50000]

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scaling features
# scaler = StandardScaler()
# X_train_scaled = scaler.fit_transform(X_train)
# X_test_scaled = scaler.transform(X_test)

# CNN Model
model = Sequential([
    Input(shape=(X_train.shape[1], 1)),
    Conv1D(32, kernel_size=3, activation='relu'),
    Dropout(0.3),
    Conv1D(64, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Dropout(0.3),
    Conv1D(128, kernel_size=3, activation='relu'),
    Dropout(0.3),
    Conv1D(256, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Dropout(0.3),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(1)
])

# Compile and train the model
model.compile(optimizer='adam', loss='mean_squared_error')
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2)

# Plot training and validation loss
plt.plot(range(1, len(history.history['loss']) + 1), history.history['loss'], 'bo', label='Training Loss')
plt.plot(range(1, len(history.history['val_loss']) + 1), history.history['val_loss'], 'r', label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

# Evaluate the model
test_predictions = model.predict(X_test)
mse = mean_squared_error(y_test, test_predictions)
r_squared = r2_score(y_test, test_predictions)
print("Mean Squared Error:", mse)
print("R-squared:", r_squared)


## Conventional Neural Network (m6a AVERAGE / STDDEV + POSITION => mRNA decay) V3

In [ ]:
model = Sequential([
    Input(shape=(X_train.shape[1], 1)),
    Conv1D(32, kernel_size=3, activation='relu'),
    Dropout(0.3),
    Conv1D(64, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Dropout(0.3),
    Conv1D(128, kernel_size=3, activation='relu'),
    Dropout(0.3),
    Conv1D(256, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Dropout(0.3),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(1)
])


In [ ]:
# Potential Future CNN Model
model = Sequential([
    Input(shape=(X_train.shape[1], 1)),
    Conv1D(64, kernel_size=5, activation='relu'),
    MaxPooling1D(pool_size=2),
    Dropout(0.3),
    Conv1D(64, kernel_size=5, activation='relu'),
    MaxPooling1D(pool_size=2),
    Dropout(0.3),
    Conv1D(64, kernel_size=5, activation='relu'),
    MaxPooling1D(pool_size=2),
    Dropout(0.3),
    Conv1D(64, kernel_size=5, activation='relu'),
    MaxPooling1D(pool_size=2),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(1)
])

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input, Conv1D, MaxPooling1D, Flatten, Dropout
import matplotlib.pyplot as plt

# Load and preprocess data
m6a_data = df_m6a.replace('NA', np.nan)
m6a_data['m6a_avg'] = m6a_data.iloc[:, 4:33].mean(axis=1)

transcriptomic_data = df_transcripts
transcriptomic_data['Transcriptomic_avg_log'] = np.log1p(transcriptomic_data.iloc[:, 2:31].mean(axis=1))

# Merge datasets
merged_data = pd.merge(m6a_data, transcriptomic_data, left_on='ENSEMBL gene', right_on='Genes')
merged_data = pd.merge(merged_data, df_sequences, on=['ENSEMBL gene', 'position gene'], how='left')
merged_data['Gene_ID'] = merged_data['ENSEMBL gene'].str.split('.', expand=True)[0]
merged_data = pd.merge(merged_data, df_half_life, left_on='Gene_ID', right_on='Ensembl Gene Id')

# Drop rows with NaN in the 'position' column
merged_data = merged_data.dropna(subset=['position'])

# Keep only the row with the largest 'position' value for each unique sequence in 'sequences_full'
merged_data = merged_data.sort_values('position', ascending=False).drop_duplicates('sequences_full')

# Define the columns to check for non-NA m6a values
m6a_columns = [
    'C11_x', 'C12_x', 'C13_x', 'C14_x', 'C15_x', 'C16_x', 'C17_x',
    'C21_x', 'C22_x', 'C23_x', 'C24_x', 'C25_x', 'C26_x', 'C27_x',
    'C31_x', 'C32_x', 'C33_x', 'C34_x', 'C35_x', 'C36_x', 'C37_x',
    'C41_x', 'C42_x', 'C43_x', 'C44_x', 'C45_x', 'C46_x', 'C47_x'
]

# Calculate Standard Deviation and Add as New Column
merged_data['m6a_stddev'] = merged_data[m6a_columns].std(axis=1)

# Filter rows with 12 or more NaN values in m6a_columns
# merged_data = merged_data.dropna(subset=m6a_columns, thresh=12)

# Prepare features and target
features = merged_data[['m6a_avg', 'position', 'sequences_full']] #
features['sequences_full'].fillna('', inplace=True)

# One-hot encoding for sequences
base_to_idx = {'A': 0, 'C': 1, 'G': 2, 'T': 3}
sequence_length = 3000
def one_hot_encode_sequence(sequence):
    encoded = np.zeros((sequence_length, 4))
    for i, base in enumerate(sequence[:sequence_length]):
        if base in base_to_idx:
            encoded[i, base_to_idx[base]] = 1
    return encoded

features['encoded_sequences'] = features['sequences_full'].apply(one_hot_encode_sequence)

# Prepare model input
X_sequence = np.array(features['encoded_sequences'].tolist())
X_m6a = features['m6a_avg'].values.reshape(-1, 1)
X_position = features['position'].values.reshape(-1, 1)
X = np.concatenate((X_sequence.reshape(X_sequence.shape[0], -1), X_m6a, X_position), axis=1) # np.concatenate(, axis=1) # , X_m6a, X_position

# Prepare target
y = merged_data['half-life (PC1)'].values

#y = merged_data['Transcriptomic_avg_log'].values

X = X[:51245]
y = y[:51245]

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scaling features
# scaler = StandardScaler()
# X_train_scaled = scaler.fit_transform(X_train)
# X_test_scaled = scaler.transform(X_test)

# Continue with the rest of your model training and evaluation
# CNN Model
model = Sequential([
    Input(shape=(X_train.shape[1], 1)),
    Conv1D(32, kernel_size=3, activation='relu'),
    Dropout(0.3),
    Conv1D(64, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Dropout(0.3),
    Conv1D(128, kernel_size=3, activation='relu'),
    Dropout(0.3),
    Conv1D(256, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Dropout(0.3),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(1)
])

# Compile and train the model
model.compile(optimizer='adam', loss='mean_squared_error')
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2)

# Plot training and validation loss
plt.plot(range(1, len(history.history['loss']) + 1), history.history['loss'], 'bo', label='Training Loss')
plt.plot(range(1, len(history.history['val_loss']) + 1), history.history['val_loss'], 'r', label='Validation Loss')
plt.title('mRNA Decay Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('MSE Loss')
plt.legend()
plt.show()

# Evaluate the model
test_predictions = model.predict(X_test)
mse = mean_squared_error(y_test, test_predictions)
r_squared = r2_score(y_test, test_predictions)
print("Mean Squared Error:", mse)
print("R-squared:", r_squared)


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input

# Assuming the previous code for data preparation and model training has been executed

# Generate ten random indices from the test set
num_predictions = 10
random_indices = np.random.choice(len(y_test), size=num_predictions, replace=False)

# Get the actual values and corresponding predictions
actual_values = y_test[random_indices]
predicted_values = test_predictions[random_indices].flatten()

# Calculate percentage error for each prediction
percentage_errors = np.abs((predicted_values - actual_values) / actual_values) * 100

# Create a DataFrame to display the information
data = {
    'Actual Values (Log Value)': actual_values,
    'Predicted Values (Log Value)': predicted_values,
    'Percentage Error (%)': percentage_errors
}
prediction_comparison = pd.DataFrame(data)

# Display the table
print(prediction_comparison)

# Calculate R-squared value
r_squared = r2_score(y_test, test_predictions)

# Scatter plot of predicted vs actual values
plt.subplot(1, 2, 2)
plt.scatter(y_test, test_predictions)
plt.plot(y_test, y_test, color='red', label='Ideal Prediction')
plt.title('mRNA Decay Test Set Predictions')
plt.xlabel('Actual Values (Log Value)')
plt.ylabel('Predicted Values (Log Value)')
plt.legend()

# Include R-squared value as text on the plot
plt.text(0.1, 0.9, f'R-squared = {r_squared:.4f}', ha='center', va='center', transform=plt.gca().transAxes, bbox=dict(facecolor='white', alpha=0.5))

plt.tight_layout()
plt.show()

## V3 HELPER CODE

In [ ]:
print(merged_data['half-life (PC1)'])

In [ ]:
print(len(set(merged_data['half-life (PC1)'])))

In [ ]:
for i in range(0,100):
  print(X_position[i])

In [ ]:
print(np.isnan(X).any(), np.isnan(y_train).any())

print(np.isinf(X).any(), np.isinf(y_train).any())

In [ ]:
import numpy as np

# Find the indices of NaN values
nan_indices = np.where(np.isnan(X[:1000]))

# Print the indices
for row, col in zip(*nan_indices):
    print(f"NaN found at Row: {row}, Column: {col}")
    print(X[row])

In [ ]:
import numpy as np

# Find the indices of NaN values
nan_indices = np.where(np.isnan(X_train))

# Print the indices
for row, col in zip(*nan_indices):
    print(f"NaN found at Row: {row}, Column: {col}")
    print(X_train[row])

## Scripts for the Data / Data Table Properties / Distributions


In [ ]:
# Print the dimensions of the DataFrame
print(f"Number of rows: {df_m6a.shape[0]}")
print(f"Number of columns: {df_m6a.shape[1]}")
print(f"Dimensions: {df_m6a.shape}")

In [ ]:
print(f"Number of rows: {df_transcripts.shape[0]}")
print(f"Number of columns: {df_transcripts.shape[1]}")
print(f"Dimensions: {df_transcripts.shape}")

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Assuming df is your DataFrame
# Replace this with your actual DataFrame
df = df_transcripts.iloc[:, 1:]

# Plot cumulative probability curves for each column on the same graph with log scale on x-axis
plt.figure(figsize=(10, 6))

for column in df.columns:
    values = df[column].dropna().sort_values()
    cumulative_prob = np.arange(len(values)) / len(values)
    plt.plot(values, cumulative_prob, label=column, marker='o', linestyle='-')

plt.xscale('log')  # Set log scale for the x-axis
plt.title('Cumulative Probability Curves for Each Sample (Log Scale)')
plt.xlabel('Values (log scale)')
plt.ylabel('Cumulative Probability')
plt.ylim(0.7, 1.0)  # Set y-axis limits
plt.legend()
plt.show()

In [ ]:
print(df_m6a)

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# Assuming your DataFrame is already loaded and named 'df'

# Drop NaN values in the 'm6a_avg' column before plotting (if needed)
m6a_avg_values = df_m6a['m6a_avg'].dropna()

# Setting style
sns.set(style="whitegrid")
plt.figure(figsize=(10, 6))

# Creating a distribution plot using seaborn
sns.histplot(m6a_avg_values, kde=True, bins=30, color='skyblue', edgecolor='black')

# Adding labels and title with improved font sizes
plt.xlabel('m6a Averages', fontsize=12)
plt.ylabel('Frequency', fontsize=12)
plt.title('Distribution of Average m6a Values', fontsize=14)

# Customize tick parameters
plt.tick_params(axis='both', which='major', labelsize=10)

# Show grid with dashed lines
plt.grid(linestyle='--', linewidth=0.5)

# Remove top and right spines
sns.despine()

# Show plot
plt.show()


In [ ]:
print(df_transcripts)

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

# Assuming your DataFrame is named df_transcripts

# Drop rows with zero Transcriptomic_avg if necessary
df_filtered = df_transcripts[df_transcripts['Transcriptomic_avg'] != 0]

# Log2 transform the 'Transcriptomic_avg' column
df_filtered['Transcriptomic_avg_log2'] = np.log2(df_filtered['Transcriptomic_avg'] + 1)  # Adding 1 to avoid log of zero

# Plotting the distribution using seaborn with log scales
sns.set(style="whitegrid")  # Setting the style for the plot
plt.figure(figsize=(8, 6))  # Setting the figure size

# Creating a histogram for the 'Transcriptomic_avg_log2' column with log scales
sns.histplot(df_filtered['Transcriptomic_avg_log2'], kde=True, bins=30, color='skyblue')

# Adding labels and title
plt.xlabel('Average Expression Log2 Values')
plt.ylabel('Frequency')
plt.title('Distribution of Log2 Transformed Expression Values')

# Displaying the plot
plt.show()


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Assuming 'half-life (PC1)' is the column name in your DataFrame df_half_life

plt.figure(figsize=(8, 6))  # Setting the figure size

# Creating a histogram using Seaborn for 'half-life (PC1)' column
sns.histplot(df_half_life['half-life (PC1)'], bins=30, color='skyblue')

# Adding labels and title
plt.xlabel('Half-life')
plt.ylabel('Frequency')
plt.title('Distribution of Half-life Values')

# Displaying the plot
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

# Assuming your DataFrame with the 'm6a_stddev' column is named merged_data

# Optionally filter the DataFrame if necessary
# For example, removing zero values if they don't make sense in your context
# merged_data_filtered = merged_data[merged_data['m6a_stddev'] != 0]

# If needed, you can apply a transformation to the data
# For this example, I'll assume no transformation is applied

# Plotting the distribution using seaborn
sns.set(style="whitegrid")  # Setting the style for the plot
plt.figure(figsize=(8, 6))  # Setting the figure size

# Creating a histogram for the 'm6a_stddev' column
sns.histplot(merged_data['m6a_stddev'], kde=True, bins=30, color='skyblue')

# Adding labels and title
plt.xlabel('Standard Deviation of m6A Values')
plt.ylabel('Frequency')
plt.title('Distribution of m6A Standard Deviation')

# Displaying the plot
plt.show()

In [ ]:
# Calculate the proportion of missing values for each column
na_proportion = df_m6a.isna().mean()

# Display the results
print("Proportion of missing values in each column:")
print(na_proportion)

In [ ]:
# Calculate the proportion of missing values for each column
na_proportion = df_transcripts.isna().mean()

# Display the results
print("Proportion of missing values in each column:")
print(na_proportion)

## INTERIM REPORT CLEANING SCRIPT (DEPRECIATED)


In [ ]:
# Remove rows with NaN values
m6a_df_cleaned = m6a_df.dropna()

# Display the cleaned DataFrame
print("Original DataFrame:")
print(m6a_df)

print("\nDataFrame after removing rows with NaN values:")
print(m6a_df_cleaned)

In [ ]:
# Remove rows with any numerical value less than twenty
df_transcripts_filtered = df_transcripts[(df >= 20).all(axis=1)]

# Display the filtered DataFrame
print("Original DataFrame:")
print(df_transcripts)

print("\nDataFrame after removing rows with any numerical value less than twenty:")
print(df_transcripts_filtered)

## INTERIM REPORT CNN (DEPRECIATED)


In [ ]:
import torch
import torch.nn as nn
import torch.utils.data
assert(torch.cuda.is_available()) # if this fails go to Runtime -> Change runtime type -> Set "Hardware Accelerator"
print("Torch version:", torch.__version__)

In [ ]:
import torch
assert(torch.cuda.is_available())
device = "cuda"

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
import numpy as np

# Assuming df_m6a and df_transcripts are your DataFrames
# Replace 'C11', 'C12', etc., with your actual column names

# Selecting specific columns
input_columns = ['C11', 'C12', 'C13', 'C14', 'C15', 'C16', 'C17','C21', 'C22', 'C23', 'C24', 'C25', 'C26', 'C27','C31', 'C32', 'C33', 'C34', 'C35', 'C36', 'C37','C41', 'C42', 'C43', 'C44', 'C45', 'C46', 'C47']  # List of input columns
output_column = 'C11'

# Randomly select 158 rows from m6a_df_cleaned
random_rows = m6a_df_cleaned.sample(n=158, random_state=42)

# Training data
input_data_train = random_rows[input_columns].values
output_data_train = df_transcripts_filtered[[output_column]].values

# Testing data
input_data_test = random_rows[input_columns].values
output_data_test = df_transcripts_filtered[[output_column]].values

# Convert to PyTorch tensors
input_tensor_train = torch.tensor(input_data_train, dtype=torch.float32)
output_tensor_train = torch.tensor(output_data_train, dtype=torch.float32).view(-1, 1)  # Reshape for single output

input_tensor_test = torch.tensor(input_data_test, dtype=torch.float32)
output_tensor_test = torch.tensor(output_data_test, dtype=torch.float32).view(-1, 1)

# Print the sizes of tensors for debugging
print("Size of input_tensor_train:", input_tensor_train.size())
print("Size of output_tensor_train:", output_tensor_train.size())

# Define the Deep Neural Network (DNN) model
class DeepDNN(nn.Module):
    def __init__(self, input_dim, hidden_dims, output_dim):
        super(DeepDNN, self).__init__()
        self.input_layer = nn.Linear(input_dim, hidden_dims[0])
        self.hidden_layers = nn.ModuleList([nn.Linear(hidden_dims[i], hidden_dims[i+1]) for i in range(len(hidden_dims)-1)])
        self.output_layer = nn.Linear(hidden_dims[-1], output_dim)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.input_layer(x))
        for layer in self.hidden_layers:
            x = self.relu(layer(x))
        x = self.output_layer(x)
        return x

# Instantiate the model, loss function, and optimizer
input_dim = len(input_columns)  # Number of input features
hidden_dims = [8, 8, 8]  # Adjust the number of neurons in each hidden layer
output_dim = 1

model = DeepDNN(input_dim, hidden_dims, output_dim)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training the model
epochs = 100
batch_size = 16

train_dataset = TensorDataset(input_tensor_train, output_tensor_train)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

for epoch in range(epochs):
    for inputs, targets in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch + 1}/{epochs}, Loss: {loss.item()}")

# Test the model on the test set
with torch.no_grad():
    model.eval()
    test_outputs = model(input_tensor_test)
    test_loss = criterion(test_outputs, output_tensor_test)
    print(f"Test Loss: {test_loss.item()}")


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
import numpy as np

# Assuming df_m6a and df_transcripts are your DataFrames
# Replace 'C11', 'C12', etc., with your actual column names

# Selecting specific columns
input_columns = ['C11', 'C12', 'C13', 'C14', 'C15', 'C16', 'C17','C21', 'C22', 'C23', 'C24', 'C25', 'C26', 'C27','C31', 'C32', 'C33', 'C34', 'C35', 'C36', 'C37','C41', 'C42', 'C43', 'C44', 'C45', 'C46', 'C47']  # List of input columns
output_columns = ['C11', 'C12', 'C13', 'C14', 'C15', 'C16', 'C17','C21', 'C22', 'C23', 'C24', 'C25', 'C26', 'C27','C31', 'C32', 'C33', 'C34', 'C35', 'C36', 'C37','C41', 'C42', 'C43', 'C44', 'C45', 'C46', 'C47']  # List of output columns

# Randomly select 158 rows from m6a_df_cleaned
random_rows = m6a_df_cleaned.sample(n=158, random_state=42)

# Training data
input_data_train = random_rows[input_columns].values
output_data_train = df_transcripts_filtered[output_columns].values

# Randomly select 158 rows from m6a_df_cleaned
random_rows = m6a_df_cleaned.sample(n=158, random_state=3)

# Testing data
input_data_test = random_rows[input_columns].values
output_data_test = df_transcripts_filtered[output_columns].values

# Convert to PyTorch tensors
input_tensor_train = torch.tensor(input_data_train, dtype=torch.float32)
output_tensor_train = torch.tensor(output_data_train, dtype=torch.float32)

input_tensor_test = torch.tensor(input_data_test, dtype=torch.float32)
output_tensor_test = torch.tensor(output_data_test, dtype=torch.float32)

# Print the sizes of tensors for debugging
print("Size of input_tensor_train:", input_tensor_train.size())
print("Size of output_tensor_train:", output_tensor_train.size())

# Define the Deep Neural Network (DNN) model
class DeepDNN(nn.Module):
    def __init__(self, input_dim, hidden_dims, output_dim):
        super(DeepDNN, self).__init__()
        self.input_layer = nn.Linear(input_dim, hidden_dims[0])
        self.hidden_layers = nn.ModuleList([nn.Linear(hidden_dims[i], hidden_dims[i+1]) for i in range(len(hidden_dims)-1)])
        self.output_layer = nn.Linear(hidden_dims[-1], output_dim)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.input_layer(x))
        for layer in self.hidden_layers:
            x = self.relu(layer(x))
        x = self.output_layer(x)
        return x

# Instantiate the model, loss function, and optimizer
input_dim = len(input_columns)  # Number of input features
hidden_dims = [128, 64, 1]  # Adjust the number of neurons in each hidden layer
output_dim = len(output_columns)  # Number of output features

model = DeepDNN(input_dim, hidden_dims, output_dim)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training the model
epochs = 100
batch_size = 16

train_dataset = TensorDataset(input_tensor_train, output_tensor_train)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

for epoch in range(epochs):
    for inputs, targets in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch + 1}/{epochs}, Loss: {loss.item()}")

# Test the model on the test set
with torch.no_grad():
    model.eval()
    test_outputs = model(input_tensor_test)
    test_loss = criterion(test_outputs, output_tensor_test)
    print(f"Test Loss: {test_loss.item()}")


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error

# Assuming df_m6a and df_transcripts are your DataFrames
# Replace 'C11', 'C12', etc., with your actual column names

# Selecting specific columns
input_columns = ['C11', 'C12', 'C13', 'C14', 'C15', 'C16', 'C17', 'C21', 'C22', 'C23', 'C24', 'C25', 'C26', 'C27', 'C31', 'C32', 'C33', 'C34', 'C35', 'C36', 'C37', 'C41', 'C42', 'C43', 'C44', 'C45', 'C46', 'C47']  # List of input columns
output_column = 'C11'

# Randomly select 158 rows from df_m6a
random_rows = m6a_df_cleaned.sample(n=158, random_state=42)

# Log-normalize the input and output data
log_normalized_train_data = np.log1p(random_rows[input_columns])
log_normalized_train_labels = np.log1p(df_transcripts_filtered[output_column])

random_rows = m6a_df_cleaned.sample(n=158, random_state=44)

log_normalized_train_data = np.log1p(random_rows[input_columns])
log_normalized_train_labels = np.log1p(df_transcripts_filtered[output_column])

# Perform Lasso regression
alpha = 0.0001 # Lasso regularization strength
lasso_model = Lasso(alpha=alpha)
lasso_model.fit(train_data, train_labels)

# Make predictions on the test set
lasso_predictions = lasso_model.predict(test_data)

# Calculate and print Mean Squared Error (MSE)
mse = mean_squared_error(test_labels, lasso_predictions)
print(f'Mean Squared Error (MSE): {mse}')

# Plotting the scatterplot
plt.figure(figsize=(10, 6))

# Scatterplot of actual vs. predicted values
plt.scatter(test_labels, lasso_predictions, color='blue', label='Actual vs. Predicted')

# Plotting the diagonal line for reference
plt.plot([min(test_labels), max(test_labels)], [min(test_labels), max(test_labels)], linestyle='--', color='red', label='Perfect Prediction')

plt.title('Lasso Regression (Log-Normalized): Actual vs. Predicted')
plt.xlabel('Log-Normalized Actual Values')
plt.ylabel('Log-Normalized Predicted Values')
plt.legend()
plt.show()